In [18]:
import copy
import urllib.request
import random
import folium
import xml.sax
from math import radians, cos, sin, asin, sqrt
from pathlib import Path
import networkx as nx
from pprint import pprint


from vars import DATASET
print(DATASET)

cambridge


In [19]:
def haversine_distance(lon1, lat1, lon2, lat2, unit_m=True):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    default unit : km
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of the Earth in kilometers. Use 3956 for miles
    if unit_m:
        r *= 1000
    return c * r


class Node(object):
    def __init__(self, id, lon, lat):
        self.id = id
        self.lon = lon
        self.lat = lat
        self.tags = {}

    def __str__(self):
        return "Node (id : %s) lon : %s, lat : %s "%(self.id, self.lon, self.lat)


class Way(object):
    def __init__(self, id, osm):
        self.osm = osm
        self.id = id
        self.nds = []
        self.tags = {}

    def split(self, dividers):
        # slice the node-array using this nifty recursive function
        def slice_array(ar, dividers):
            for i in range(1,len(ar)-1):
                if dividers[ar[i]]>1:
                    left = ar[:i+1]
                    right = ar[i:]

                    rightsliced = slice_array(right, dividers)

                    return [left]+rightsliced
            return [ar]

        slices = slice_array(self.nds, dividers)

        # create a way object for each node-array slice
        ret = []
        i = 0
        for slice in slices:
            littleway = copy.copy(self)
            littleway.id += "-%d" % i
            littleway.nds = slice
            ret.append(littleway)
            i += 1

        return ret



In [20]:
class OSM(object):
    def __init__(self, osm_xml_data, is_xml_string=True):
        """ File can be either a filename or stream/file object.

        set `is_xml_string=False` if osm_xml_data is a filename or a file stream.
        """
        nodes = {}
        ways = {}

        superself = self

        class OSMHandler(xml.sax.ContentHandler):
            @classmethod
            def setDocumentLocator(self, loc):
                pass

            @classmethod
            def startDocument(self):
                pass

            @classmethod
            def endDocument(self):
                pass

            @classmethod
            def startElement(self, name, attrs):
                if name == 'node':
                    self.currElem = Node(attrs['id'], float(attrs['lon']), float(attrs['lat']))
                elif name == 'way':
                    self.currElem = Way(attrs['id'], superself)
                elif name == 'tag':
                    self.currElem.tags[attrs['k']] = attrs['v']
                elif name == 'nd':
                    self.currElem.nds.append(attrs['ref'])

            @classmethod
            def endElement(self, name):
                if name == 'node':
                    nodes[self.currElem.id] = self.currElem
                elif name == 'way':
                    ways[self.currElem.id] = self.currElem

            @classmethod
            def characters(self, chars):
                pass

        if is_xml_string:
            xml.sax.parseString(osm_xml_data, OSMHandler)
        else:
            with open(osm_xml_data, mode='r') as f:
                xml.sax.parse(f, OSMHandler)

        self.nodes = nodes
        self.ways = ways

        # count times each node is used
        node_histogram = dict.fromkeys(self.nodes.keys(), 0)
        for way in self.ways.values():
            if len(way.nds) < 2:  # if a way has only one node, delete it out of the osm collection
                del self.ways[way.id]
            else:
                for node in way.nds:
                    node_histogram[node] += 1

        # use that histogram to split all ways, replacing the member set of ways
        new_ways = {}
        for id, way in self.ways.items():
            split_ways = way.split(node_histogram)
            for split_way in split_ways:
                new_ways[split_way.id] = split_way
        self.ways = new_ways


In [35]:
def read_osm(osm_xml_data, is_xml_string=True, only_roads=True):
    """Read graph in OSM format from file specified by name or by stream object.
    Parameters
    ----------
    filename_or_stream : filename or stream object

    Returns
    -------
    G : Graph

    Examples
    --------
    >>> G=nx.read_osm(nx.download_osm(-122.33,47.60,-122.31,47.61))
    >>> import matplotlib.pyplot as plt
    >>> plt.plot([G.node[n]['lat']for n in G], [G.node[n]['lon'] for n in G], 'o', color='k')
    >>> plt.show()
    """
    osm = OSM(osm_xml_data, is_xml_string=is_xml_string)
    G = nx.DiGraph()

    ## Add ways
    for w in osm.ways.values():
        if only_roads and 'highway' not in w.tags:
            continue
          
        if ('oneway' in w.tags):
            if (w.tags['oneway'] == 'yes'):
                # ONLY ONE DIRECTION
                nx.add_path(G, w.nds, id=w.id, tags=w.tags)
            else:
                # BOTH DIRECTION
                nx.add_path(G, w.nds, id=w.id, tags=w.tags)
                nx.add_path(G, w.nds[::-1], id=w.id, tags=w.tags)        
                
        else:
            # BOTH DIRECTION
            nx.add_path(G, w.nds, id=w.id, tags=w.tags)
            nx.add_path(G, w.nds[::-1], id=w.id, tags=w.tags)
#             nx.add_path(G, w.nds, id=w.id)
#             nx.add_path(G, w.nds[::-1], id=w.id)

    # Complete the used nodes' information
    coordinates_map = {}
    for n_id in G.nodes():
        n = osm.nodes[n_id]
        G.nodes[n_id]['lat'] = n.lat
        G.nodes[n_id]['lon'] = n.lon
        G.nodes[n_id]['id'] = n.id
        G.nodes[n_id]['tags'] = n.tags
        coordinates_map[n_id] = (n.lon, n.lat)

    # Estimate the length of each way
    for u, v, d in G.edges(data=True):
        # Give a realistic distance estimation (neither EPSG nor projection nor reference system are specified)
        distance = haversine_distance(G.nodes[u]['lon'], G.nodes[u]['lat'], G.nodes[v]['lon'], G.nodes[v]['lat'], unit_m=True)

        G.add_weighted_edges_from([(u, v, distance)], weight='havlen')

#     G = nx.relabel_nodes(G, coordinates_map)
    return G


In [36]:
g = read_osm("./" + DATASET + ".osm", is_xml_string=False)
# make it undirected, because directed just isn't worth the trouble yet
# g = g.to_undirected()
len(g)

21438

# Delete valence two (in-between) nodes

### One-Way Streets

In [ ]:
# deal with valence 2 nodes (intermediaries on one way streets)

# nodes with degree two. i.e. they are in between going from one place to another.
valence_2_nodes = set([k for k, v in g.degree() if v == 2])
print('original len valence_2_nodes: ' + str(len(valence_2_nodes)))

def right_node(node):
    print('right-node:' + node)
    out_edges = [x for x in g.out_edges(node)]
    
    if len(out_edges) > 0:
        return [x for x in g.out_edges(node)][0][1]
        
    else:
        return ''

def left_node(node):
    print('left-node:' + node)
    in_edges = [x for x in g.in_edges(node)]
    if len(in_edges) > 0:
        return [x for x in g.in_edges(node)][0][0]
    else:
        return ''


newly_linked = {}

while len(valence_2_nodes) > 0:
    node = list(valence_2_nodes)[0]

    old_right, new_right = node, right_node(node)
    old_left, new_left = node, left_node(node)
    nodes_for_deletion = [node]
    distance = 0

    # find leftmost vertex with valence 2
    while new_left in valence_2_nodes:
        distance += g.get_edge_data(new_left, old_left).get('havlen', 0) # add distance
        old_left = new_left # re-assign the rightmost valence 2 node.
        nodes_for_deletion.insert(0, old_left) # mark the rightmost v2 node for deletion
        new_left = left_node(old_left) # reassign the rightmost node of questionable valence

    # add distance to new leftmost valence != 2 node
    if new_left != '':
        distance += g.get_edge_data(new_left, old_left).get('havlen', 0)

    # find rightmost vertex with valence 2
    while new_right in valence_2_nodes:
        distance += g.get_edge_data(old_right, new_right).get('havlen', 0)
        old_right = new_right
        nodes_for_deletion.append(old_right)
        new_right = right_node(old_right)

    # add distance to new rightmost valence != 2 node, if a more rightmost node exists
    if new_right != '':
        distance += g.get_edge_data(old_right, new_right).get('havlen', 0)
       
    # remove old useless nodes
    g.remove_nodes_from(nodes_for_deletion)
    valence_2_nodes -= set(nodes_for_deletion)
    
    # add new edge between end nodes
    g.add_weighted_edges_from([(new_left, new_right, distance)], weight='havlen')
    
    newly_linked[(new_left, new_right)] = [nodes_for_deletion]


print('num_valence_2_nodes remaining: ' + str(len(valence_2_nodes)) + "(should be 0)")
# pprint(newly_linked)

In [24]:
node = '61438866'
g.out_edges(node)

OutEdgeDataView([])

In [26]:
[x for x in g.out_edges(node)]

[]

In [25]:
g.in_edges(node)

InEdgeDataView([('1519077271', '61438866'), ('3605917788', '61438866')])

## bi-directional streets

In [11]:
# valence_4_nodes = set([k for k, v in g.degree() if v == 4])
# special_node = '61331813'
# for node in [special_node]:
#     print(node)
#     print(g.nodes[node])
#     print(g.out_edges(node))
#     print(g.in_edges(node))
    
# #     [item for sublist in l for item in sublist]
#     out_edges = set([item for sublist in g.out_edges(node) for item in sublist]) - set([node])
#     in_edges = set([item for sublist in g.in_edges(node) for item in sublist]) - set([node])
#     print(out_edges)
#     print(in_edges)
    
    
#     print('----')


In [12]:
from itertools import compress 
    
def intermediary_two_way_node(node):
    '''returns true if the given node is intermediary on a two way street.
       returns false if given node is not intermediary on a two way street.
       errors if node does not exist.'''
    out_edges = set([item for sublist in g.out_edges(node) for item in sublist]) - set([node])
    in_edges = set([item for sublist in g.in_edges(node) for item in sublist]) - set([node])

    if len(out_edges) == 2 and len(in_edges) == 2:
        return in_edges == out_edges
    else:
        return False

def right_two_way_node(node):
    if intermediary_two_way_node(node):
        return [x for x in g.out_edges(node)][1][1]
    else:
        return False

def left_two_way_node(node):
    if intermediary_two_way_node(node):
        return [x for x in g.out_edges(node)][0][1]
    else:
        return False
    
def compute_distances_of_walk(walk_list):
    '''given a set of walking nodes, Compute the distance between them.'''
    distance = 0
    start_node = walk_list[0]
    
    for next_node in walk_list[1:]:
        sprint = g.get_edge_data(start_node, next_node).get('havlen', 0)
        distance += sprint
        start_node = next_node
    return distance


def walk_right(node):
    if not intermediary_two_way_node(node):
        return {'walked': [node],
                'distance': 0}
    
    # compute node sequence
    walked = [node]
    next_node = right_two_way_node(node)
    
    while intermediary_two_way_node(next_node) and (next_node not in walked):
        walked.append(next_node)
        next_node = right_two_way_node(next_node)
      
    # append non-intermediary node
    walked.append(next_node)
    
    # compute distances    
    return {'walked': walked,
            'distance': compute_distances_of_walk(walked)}

def walk_left(node):
    return {'walked': [node],
            'distance': 0}
    
    walked = [node]
    next_node = left_two_way_node(node)
    
    while intermediary_two_way_node(next_node) and (next_node not in walked):
        walked.append(next_node)
        next_node = left_two_way_node(next_node)
    
    # append non-intermediary node
    walked.append(next_node)
    
    return {'walked': walked,
            'distance': compute_distances_of_walk(walked)}

    

In [13]:
special_node = '61331813'
walk_right(special_node)

{'walked': ['61331813'], 'distance': 0}

In [14]:


# get the intermediate nodes
intermediary_2w = set([x for x in g.nodes() if intermediary_two_way_node(x)])

# walk the grah, and remove intermediary nodes
while len(intermediary_2w) > 0:        
    node = list(intermediary_2w)[0]
#     print(node)

        
    right = walk_right(node)
    left = walk_left(node)
    
    # all walked nodes (use a set to eliminate duplicate start node)
    all_walked = set(left['walked'] + right['walked'])
    
    # the total distance walked
    all_distance = left['distance'] + right['distance']
    
    # modify graph
    # Add new edge between non-intermediary nodes
    g.add_weighted_edges_from([(left['walked'][-1], right['walked'][-1], all_distance)], weight='havlen')
    # delete intermediary nodes
    nodes_for_deletion = set(left['walked'][:-1] + right['walked'][:-1])
    g.remove_nodes_from(nodes_for_deletion)
    
    # modify intermediary list
    print(len(intermediary_2w))
    intermediary_2w -= nodes_for_deletion
    
    # edge case with single node cycle
    if len(nodes_for_deletion) == 0:
        intermediary_2w -= set([node])
        
    print(len(intermediary_2w)) 
    
    
#     print(node)
#     print(nodes_for_deletion)
#     print(all_walked)
#     print(all_distance)
    print('---')
    
    

2693195879
1281
1280
---
5458232831
1280
1279
---
7629147370
1279
1278
---
61322508
1278
1276
---
6755415499
1276
1275
---
6594977027
1275
1274
---
1705793314
1274
1273
---
4039158116
1273
1272
---
7629493606
1272
1271
---
4330221296
1271
1270
---
3616211886
1270
1268
---
4633013924
1268
1267
---
1705790258
1267
1266
---
6877670837
1266
1265
---
61397786
1265
1263
---
7631664252
1263
1262
---
5458857447
1262
1261
---
3687638510
1261
1259
---
6755418760
1259
1257
---
314279338
1257
1256
---
7631495293
1256
1255
---
1548276661
1255
1253
---
1525366585
1253
1252
---
7598963603
1252
1251
---
3809869816
1251
1250
---
4804188849
1250
1248
---
3252968777
1248
1247
---
3620902824
1247
1245
---
7626171461
1245
1244
---
61321511
1244
1243
---
7631487175
1243
1242
---
5458858159
1242
1241
---
7505423252
1241
1240
---
7629147337
1240
1239
---
6309394726
1239
1237
---
575444717
1237
1234
---
1469359665
1234
1233
---
7631668538
1233
1232
---
4330616518
1232
1231
---
4040438294
1231
1230
---
72335859

7802219139
763
762
---
5458226293
762
761
---
61322550
761
759
---
61324758
759
758
---
7598963513
758
757
---
7641785908
757
756
---
7626165530
756
754
---
2060282039
754
753
---
1708499157
753
752
---
61318134
752
751
---
7579360334
751
750
---
61327632
750
749
---
6297996785
749
748
---
5035889428
748
747
---
380881055
747
745
---
4978078197
745
743
---
7631668574
743
742
---
7628786986
742
741
---
7628787325
741
740
---
5458232821
740
739
---
7628877227
739
738
---
7703039126
738
737
---
597845245
737
736
---
4330221309
736
734
---
7629147480
734
733
---
7802026297
733
732
---
986189403
732
731
---
7641829757
731
730
---
61330557
730
729
---
7631664255
729
728
---
1830019665
728
727
---
7628787069
727
726
---
7629493658
726
725
---
61397687
725
724
---
6755315137
724
723
---
7631487140
723
722
---
7629493815
722
721
---
7631668815
721
719
---
7629494030
719
717
---
7629493671
717
715
---
7629493777
715
714
---
61332076
714
713
---
1818119326
713
712
---
7631668717
712
711
---
54588

297
296
---
380881219
296
295
---
7598963503
295
294
---
6309394740
294
293
---
7629147346
293
292
---
7629147392
292
291
---
2507563251
291
290
---
688404589
290
289
---
7626171606
289
288
---
575445941
288
287
---
7631495262
287
286
---
7629493893
286
285
---
5458847024
285
284
---
82752724
284
283
---
383976734
283
282
---
7628787259
282
281
---
3616211891
281
280
---
6668894105
280
279
---
7629493991
279
278
---
7631668895
278
277
---
1548276791
277
276
---
7727679868
276
275
---
7771291229
275
274
---
7727679836
274
273
---
314502411
273
272
---
7631668517
272
271
---
7628787101
271
270
---
7626171557
270
269
---
7628787017
269
268
---
7634532980
268
267
---
5737576871
267
266
---
7628787225
266
265
---
7628856064
265
264
---
5562638423
264
263
---
315559750
263
262
---
5458828848
262
261
---
5458235167
261
260
---
7626171410
260
259
---
7626171416
259
258
---
7658055840
258
257
---
1548276337
257
256
---
7628787117
256
255
---
7794600747
255
254
---
7569985113
254
253
---
5458857

In [15]:
len(g.nodes())

10070

# Write to Disk

In [16]:
nx.write_gpickle(g, "01_" + DATASET + ".gpickle")

In [17]:
# export to graphml. graphml export can't handle some datatypes (including dicts and None)
# so delete them
h = g.copy()

for n in h.nodes():
    h.nodes[n].pop('tags', None)
for e in g.edges():
    h.edges[e].pop('tags', None)
nx.write_graphml(h, "01_" + DATASET + ".graphml")

# clear up memory
h = None

# Cruft

## Dictionary of Tags

In [51]:
tags = {}
osm = OSM('./' + DATASET + '.osm', is_xml_string=False)

for w in osm.ways.values():
    for t in w.tags:
        if tags.get(t):
            tags[t] = tags[t] + 1
        else:
            tags[t] = 1

In [52]:
tags

{'attribution': 146,
 'condition': 141,
 'highway': 205,
 'lanes': 142,
 'massgis:way_id': 142,
 'maxspeed': 107,
 'name': 169,
 'oneway': 108,
 'source': 154,
 'surface': 126,
 'width': 131,
 'cycleway:right': 15,
 'massgis:ref': 16,
 'parking:lane:left': 10,
 'lit': 3,
 'foot': 12,
 'access': 13,
 'building': 643,
 'addr:housenumber': 508,
 'addr:street': 507,
 'amenity': 17,
 'brand': 5,
 'brand:wikidata': 4,
 'brand:wikipedia': 4,
 'dispensing': 3,
 'drive_through': 3,
 'healthcare': 3,
 'opening_hours': 6,
 'payment:cash': 3,
 'payment:visa': 3,
 'phone': 7,
 'website': 10,
 'wheelchair': 3,
 'addr:city': 10,
 'addr:postcode': 11,
 'building:levels': 8,
 'operator': 3,
 'power': 3,
 'ref': 2,
 'substation': 1,
 'voltage': 3,
 'addr:state': 5,
 'shop': 2,
 'denomination': 3,
 'religion': 3,
 'area': 8,
 'created_by': 6,
 'leisure': 20,
 'massgis:ARTICLE97': 8,
 'massgis:ASSESS_ACR': 8,
 'massgis:ATT_DATE': 8,
 'massgis:DCAM_ID': 8,
 'massgis:DEED_ACRES': 8,
 'massgis:EOEAINVOLV': 8